<a href="https://colab.research.google.com/github/svarogjk/shop_revenue_prediction/blob/master/gstore_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Readings

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
current_palette = sns.color_palette()
# from tqdm import tqdm_notebook
%matplotlib inline 
import io
from google.colab import files

In [0]:
import json
from pandas.io.json import json_normalize
import random

In [0]:
from itertools import product
from sklearn.preprocessing import LabelEncoder

In [6]:
!pip install fbprophet
from fbprophet import Prophet

In [0]:
from xgboost import XGBRegressor
from xgboost import plot_importance

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import r2_score, make_scorer

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
file_list = drive.ListFile({'q': "'19ry360g_dRMf_i9Gi4A2WrOzE682GA7O' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

title: gstore_competition.ipynb, id: 1gRRqwzC-VDFe_fOQW9Kc-5YX3Gmofxnt
title: train.csv, id: 1-9OX7E7uSovNTI4-h43oMQpZavJai6sb
title: test.csv, id: 1SohvLEveFD4yhhR_alpJtfgtY5QIYMV4
title: sample_submission.csv, id: 1WLB9e5uswsrR8NWq4yLwFKlXE9uYUZKB
title: all.zip (Unzipped Files), id: 13XaW-vTIJKNDKMrbhamaduANnnPFojkQ
title: all.zip, id: 1Mi8GWSji2h1FGzWkQHYXmRuQNqHcIUlj


In [11]:
train = drive.CreateFile({'id': '1-9OX7E7uSovNTI4-h43oMQpZavJai6sb'})
train.GetContentFile('train.csv')

test_data = drive.CreateFile({'id': '1SohvLEveFD4yhhR_alpJtfgtY5QIYMV4'})
test_data.GetContentFile('test.csv')

submission = drive.CreateFile({'id': '1WLB9e5uswsrR8NWq4yLwFKlXE9uYUZKB'})
test_data.GetContentFile('sample_submission.csv')

INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v2/files/1-9OX7E7uSovNTI4-h43oMQpZavJai6sb?alt=json
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v2/files/1SohvLEveFD4yhhR_alpJtfgtY5QIYMV4?alt=json


### Here we read our DataFrame and transform its json columns into normal ones

In [0]:
def json_read(df):
    
    columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    p = 0.07
    
    data_frame = df
    
    #Importing the dataset
    df = pd.read_csv(data_frame, 
                     converters={column: json.loads for column in columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'},
                    skiprows=lambda i: i>0 and random.random() > p
                    ) # transforming this column to string
    
    for column in columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    # Printing the shape of dataframes that was imported     
    print(f"Loaded {os.path.basename(data_frame)}. Shape: {df.shape}")
    return df # returning the df after importing and transforming

In [16]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

train = json_read('train.csv')
test_data = json_read('test.csv')
submission = json_read('sample_submission.csv')

Loaded train.csv. Shape: (63195, 24)
Loaded test.csv. Shape: (56363, 24)
Loaded sample_submission.csv. Shape: (56235, 24)


In [17]:
train.fullVisitorId[:2]

0    3696906537737368442
1    3982295596181714479
Name: fullVisitorId, dtype: object

In [18]:
train.columns
test_data.columns
submission.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.device', 'geoNetwork.geoNetwork', 'totals.totals',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [19]:
test_data.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.device', 'geoNetwork.geoNetwork', 'totals.totals',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [21]:
submission.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.device', 'geoNetwork.geoNetwork', 'totals.totals',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [20]:
train.head(2)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,3696906537737368442,3696906537737368442_1472856874,Not Socially Engaged,1472856874,1,1472856874,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Sou...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,3982295596181714479,3982295596181714479_1472825889,Not Socially Engaged,1472825889,1,1472825889,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Eastern...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [41]:
train.trafficSource.unique()

array(['{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "(not provided)", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}}',
       '{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "google + online", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}}',
       '{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "(not provided)", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}, "isTrueDirect": true}',
       ...,
       '{"campaign": "AW - Dynamic Search Ads Whole Site", "source": "google", "medium": "cpc", "keyword": "6qEhsCssdK0z36ri", "adwordsClickInfo": {"page": "1", "slot": "Top", "criteriaParameters": "not available in demo dataset", "gclId": "CPi135HaqdECFYtKDQodGWYG0g", "adNetworkType": "Google Search", "isVideoAd": false}, "isTrueDirect": true}',
       '{"campaign": "AW - Dynamic Search Ads Whole Site", "source"

In [24]:
test_data.head(1)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,2539146239106597280,2539146239106597280_1508207776,Not Socially Engaged,1508207776,1,1508207776,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [25]:
submission.head(2)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6097905367189223328,6097905367189223328_1508218860,Not Socially Engaged,1508218860,1,1508218860,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Direct,20171016,6607438459060540448,6607438459060540448_1508179515,Not Socially Engaged,1508179515,1,1508179515,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",...,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)


In [27]:
test_data.shape

(56363, 24)

In [23]:
train.shape

(63195, 24)

In [28]:
submission.shape

(56235, 24)

In [30]:
train_sample = train[:5].copy().T
train_sample

,0,1,2,3,4
channelGrouping,Organic Search,Organic Search,Organic Search,Organic Search,Organic Search
date,20160902,20160902,20160902,20160902,20160902
fullVisitorId,3696906537737368442,3982295596181714479,1397786681184924394,1381975521299261523,0715569826025005549
sessionId,3696906537737368442_1472856874,3982295596181714479_1472825889,1397786681184924394_1472822131,1381975521299261523_1472829727,0715569826025005549_1472864193
socialEngagementType,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
visitId,1472856874,1472825889,1472822131,1472829727,1472864193
visitNumber,1,1,1,1,1
visitStartTime,1472856874,1472825889,1472822131,1472829727,1472864193
device.device,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'browser': 'Chrome', 'browserVersion': 'not a..."
geoNetwork.geoNetwork,"{'continent': 'Americas', 'subContinent': 'Sou...","{'continent': 'Asia', 'subContinent': 'Eastern...","{'continent': 'Europe', 'subContinent': 'North...","{'continent': 'Asia', 'subContinent': 'Eastern...","{'continent': 'Americas', 'subContinent': 'Sou..."


In [38]:
df.shape

(63195, 6)

In [37]:
df.isna().sum()

bounces               31620
hits                      0
newVisits             14090
pageviews                 8
transactionRevenue    62376
visits                    0
dtype: int64

In [40]:
test_data.head(2)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,2539146239106597280,2539146239106597280_1508207776,Not Socially Engaged,1508207776,1,1508207776,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20171016,8375379770594309032,8375379770594309032_1508213189,Not Socially Engaged,1508213189,1,1508213189,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [41]:
test_data["totals.totals"][:5]

0    {'visits': '1', 'hits': '3', 'pageviews': '3',...
1    {'visits': '1', 'hits': '4', 'pageviews': '4',...
2    {'visits': '1', 'hits': '4', 'pageviews': '4',...
3    {'visits': '1', 'hits': '4', 'pageviews': '4',...
4    {'visits': '1', 'hits': '5', 'pageviews': '5',...
Name: totals.totals, dtype: object

In [43]:
df = train["totals.totals"].apply(pd.Series)
df.head(2)

,bounces,hits,newVisits,pageviews,transactionRevenue,visits
0,1,1,1,1,NaN,1
1,1,1,1,1,NaN,1


In [42]:
df_test = test_data["totals.totals"].apply(pd.Series)
df_test.head(2)

,bounces,hits,newVisits,pageviews,visits
0,NaN,3,1,3,1
1,NaN,4,1,4,1


In [44]:
df_sub = submission["totals.totals"].apply(pd.Series)
df_sub.head(2)

,bounces,hits,newVisits,pageviews,visits
0,NaN,3,1,3,1
1,NaN,3,1,3,1
